In [ ]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

In [2]:
proj_dir = Path("../../../")

In [3]:
mcnary_fish_df = pd.read_csv(proj_dir / "Data/insitu/fish/processed/DART_MCN.csv")
priest_rapids_fish_df = pd.read_csv(proj_dir / "Data/insitu/fish/processed/DART_PRD.csv")
prosser_fish_df = pd.read_csv(proj_dir / "Data/insitu/fish/processed/DART_PRO.csv")
ice_harbor_fish_df = pd.read_csv(proj_dir / "Data/insitu/fish/processed/DART_IHR.csv")

mcnary_fish_df["date"] = pd.to_datetime(mcnary_fish_df["date"])
priest_rapids_fish_df["date"] = pd.to_datetime(priest_rapids_fish_df["date"])
prosser_fish_df["date"] = pd.to_datetime(prosser_fish_df["date"])
ice_harbor_fish_df["date"] = pd.to_datetime(ice_harbor_fish_df["date"])

mcnary_fish_df['year'] = mcnary_fish_df['date'].dt.year
priest_rapids_fish_df['year'] = priest_rapids_fish_df['date'].dt.year
prosser_fish_df['year'] = prosser_fish_df['date'].dt.year
ice_harbor_fish_df['year'] = ice_harbor_fish_df['date'].dt.year

mcnary_fish_df['month'] = mcnary_fish_df['date'].dt.month
priest_rapids_fish_df['month'] = priest_rapids_fish_df['date'].dt.month
prosser_fish_df['month'] = prosser_fish_df['date'].dt.month
ice_harbor_fish_df['month'] = ice_harbor_fish_df['date'].dt.month

mcnary_fish_df['day'] = mcnary_fish_df['date'].dt.day
priest_rapids_fish_df['day'] = priest_rapids_fish_df['date'].dt.day
prosser_fish_df['day'] = prosser_fish_df['date'].dt.day
ice_harbor_fish_df['day'] = ice_harbor_fish_df['date'].dt.day

In [4]:
run_schedules = {
    "mcnary": {'spring': ['04-01', '06-08'], 'summer': ['06-09', '08-08'], 'fall': ['08-09', '10-31']},
    "priest_rapids": {'spring': ['04-15', '06-13'], 'summer': ['06-14', '08-13'], 'fall': ['08-14', '11-15']},
    "prosser": {'spring': ['03-01', '08-15'], 'fall': ['08-16', '12-15']},
    "ice_harbor": {'spring': ['04-01', '06-11'], 'summer': ['06-12', '08-11'], 'fall': ['08-12', '12-15']}
}

In [5]:
# allocate run schedules to each fish data frame
def allocate_runs(df, run_schedule):
    df_copy = df.copy()
    df_copy['year'] = df_copy['date'].dt.year
    for run, dates in run_schedule.items():
        for year in df_copy["year"].unique():
            start_date = pd.Timestamp(f"{year}-{dates[0]}")
            end_date = pd.Timestamp(f"{year}-{dates[1]}")
            df_copy.loc[
                (df_copy["date"] >= start_date) & (df_copy["date"] <= end_date), "run"
            ] = run
        df_copy = pd.concat([df_copy, pd.DataFrame(
            {
                "run": run,
                "year": df_copy["year"].unique(),
            }
        )])
    return df_copy

In [ ]:
allocate_runs(mcnary_fish_df, run_schedules["mcnary"])

In [7]:
mcnary_fish_df = allocate_runs(mcnary_fish_df, run_schedules["mcnary"])
priest_rapids_fish_df = allocate_runs(priest_rapids_fish_df, run_schedules["priest_rapids"])
prosser_fish_df = allocate_runs(prosser_fish_df, run_schedules["prosser"])
ice_harbor_fish_df = allocate_runs(ice_harbor_fish_df, run_schedules["ice_harbor"])

In [8]:
def plot_stacked_run_counts(df, ax, title, species="chinook"):

    bottom = np.zeros(len(df["year"].unique()))
    for run, group in df.fillna(0).groupby("run"):
        if run != 0:
            year_group = group.groupby("year")[species].sum().reset_index()
            # fig, ax = plt.subplots()
            p = ax.bar(
                year_group["year"], year_group[species], label=run, bottom=bottom
            )
            # print(group.groupby("year")[species].sum().reset_index())
            bottom += group.groupby("year")[species].sum()
    ax.set_title(title)
    ax.legend()

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(16, 7))
plot_stacked_run_counts(mcnary_fish_df, axs[0, 0], "[4] McNary Dam")
plot_stacked_run_counts(priest_rapids_fish_df, axs[0, 1], "[5] Priest Rapids Dam")
plot_stacked_run_counts(prosser_fish_df, axs[1, 0], "[16] Prosser")
plot_stacked_run_counts(ice_harbor_fish_df, axs[1, 1], "[12] Ice Harbor Dam")

axs[0, 0].set_ylabel("Chinook Count")
axs[1, 0].set_ylabel("Chinook Count")

axs[1, 0].set_xlabel("Year")
axs[1, 1].set_xlabel("Year")

fig.tight_layout()

# Chinook population in the Hanford Reach

Using the concept of control volume to assess the adult migration and population in the Hanford reach.  
Adult Chinook entering the reach:  
* McNary Dam: $Ch_{4}$

Adult Chinook exiting the reach:  
* Priest Rapids Dam: $Ch_{5}$
* Ice Harbor Dam: $Ch_{12}$
* Proser: $Ch_{16}$

Adult Chinook population in the reach:
* Hanford Reach: $Ch_{hr}$

Change in adult chinook population in the reach ($\Delta Ch_{hr}$):  
* $\Delta Ch_{hr} = Ch_{4} - Ch_{5} - Ch_{12} - Ch_{16}$

In [10]:
mcnary_chinook_df = mcnary_fish_df[['date', 'year', 'month', 'day', 'chinook', 'run', 'avg_temp(C)']]
priest_rapids_chinook_df = priest_rapids_fish_df[['date', 'year', 'month', 'day', 'chinook', 'run', 'avg_temp(C)']]
prosser_chinook_df = prosser_fish_df[['date', 'year', 'month', 'day', 'chinook', 'run']]
ice_harbor_chinook_df = ice_harbor_fish_df[['date', 'year', 'month', 'day', 'chinook', 'run', 'avg_temp(C)']]

In [11]:
yearly_total_chinook = mcnary_chinook_df.groupby('year')['chinook'].sum().reset_index().rename(columns={'chinook': 'mcnary'}).merge(
    priest_rapids_chinook_df.groupby('year')['chinook'].sum().reset_index().rename(columns={'chinook': 'priest_rapids'}),
    on='year'
).merge(
    prosser_chinook_df.groupby('year')['chinook'].sum().reset_index().rename(columns={'chinook': 'prosser'}),
    on='year'
).merge(
    ice_harbor_chinook_df.groupby('year')['chinook'].sum().reset_index().rename(columns={'chinook': 'ice_harbor'}),
    on='year'
)
yearly_total_chinook = yearly_total_chinook[(yearly_total_chinook['year'] >= 2000) & (yearly_total_chinook['year'] <= 2022)]

In [12]:
yearly_total_chinook['delta'] = yearly_total_chinook['mcnary'] - yearly_total_chinook['priest_rapids'] - yearly_total_chinook['prosser'] - yearly_total_chinook['ice_harbor']
yearly_total_chinook['percent_delta'] = yearly_total_chinook['delta'] / yearly_total_chinook['mcnary']

In [ ]:
fig, ax = plt.subplots()
ax.plot(yearly_total_chinook['year'], yearly_total_chinook['mcnary'], label='McNary')
# ax.plot(yearly_total_chinook['year'], yearly_total_chinook['priest_rapids'], label='Priest Rapids')
# ax.plot(yearly_total_chinook['year'], yearly_total_chinook['prosser'], label='Prosser')
# ax.plot(yearly_total_chinook['year'], yearly_total_chinook['ice_harbor'], label='Ice Harbor')
ax.plot(yearly_total_chinook['year'], yearly_total_chinook['delta'], label='Delta')

ax.legend()

In [ ]:
fig, ax = plt.subplots()
ax.bar(yearly_total_chinook['year'], yearly_total_chinook['percent_delta'])

In [15]:
yearly_spring_chinook = mcnary_chinook_df[(mcnary_chinook_df['run'] == 'spring')].groupby('year')['chinook'].sum().reset_index().rename(columns={'chinook': 'mcnary'}).merge(
    priest_rapids_chinook_df[(priest_rapids_chinook_df['run'] == 'spring')].groupby('year')['chinook'].sum().reset_index().rename(columns={'chinook': 'priest_rapids'}),
    on='year', how='outer'
).merge(
    prosser_chinook_df[(prosser_chinook_df['run'] == 'spring')].groupby('year')['chinook'].sum().reset_index().rename(columns={'chinook': 'prosser'}),
    on='year', how='outer'
).merge(
    ice_harbor_chinook_df[(ice_harbor_chinook_df['run'] == 'spring')].groupby('year')['chinook'].sum().reset_index().rename(columns={'chinook': 'ice_harbor'}),
    on='year', how='outer'
)



yearly_spring_chinook = yearly_spring_chinook[(yearly_spring_chinook['year'] >= 2000) & (yearly_spring_chinook['year'] <= 2022)]
yearly_spring_chinook['delta'] = yearly_spring_chinook['mcnary'] - yearly_spring_chinook['priest_rapids'] - yearly_spring_chinook['prosser'] - yearly_spring_chinook['ice_harbor']
yearly_spring_chinook['percent_delta'] = yearly_spring_chinook['delta'] / yearly_spring_chinook['mcnary']

In [16]:
yearly_summer_chinook = mcnary_chinook_df[(mcnary_chinook_df['run'] == 'summer')].groupby('year')['chinook'].sum().reset_index().rename(columns={'chinook': 'mcnary'}).merge(
    priest_rapids_chinook_df[(priest_rapids_chinook_df['run'] == 'summer')].groupby('year')['chinook'].sum().reset_index().rename(columns={'chinook': 'priest_rapids'}),
    on='year', how='outer'
).merge(
    ice_harbor_chinook_df[(ice_harbor_chinook_df['run'] == 'summer')].groupby('year')['chinook'].sum().reset_index().rename(columns={'chinook': 'ice_harbor'}),
    on='year', how='outer'
)

yearly_summer_chinook = yearly_summer_chinook[(yearly_summer_chinook['year'] >= 2000) & (yearly_summer_chinook['year'] <= 2022)]
yearly_summer_chinook['delta'] = yearly_summer_chinook['mcnary'] - yearly_summer_chinook['priest_rapids'] - yearly_summer_chinook['ice_harbor']
yearly_summer_chinook['percent_delta'] = yearly_summer_chinook['delta'] / yearly_summer_chinook['mcnary']

In [17]:
yearly_fall_chinook = mcnary_chinook_df[(mcnary_chinook_df['run'] == 'fall')].groupby('year')['chinook'].sum().reset_index().rename(columns={'chinook': 'mcnary'}).merge(
    priest_rapids_chinook_df[(priest_rapids_chinook_df['run'] == 'fall')].groupby('year')['chinook'].sum().reset_index().rename(columns={'chinook': 'priest_rapids'}),
    on='year', how='outer'
).merge(
    prosser_chinook_df[(prosser_chinook_df['run'] == 'fall')].groupby('year')['chinook'].sum().reset_index().rename(columns={'chinook': 'prosser'}),
    on='year', how='outer'
).merge(
    ice_harbor_chinook_df[(ice_harbor_chinook_df['run'] == 'fall')].groupby('year')['chinook'].sum().reset_index().rename(columns={'chinook': 'ice_harbor'}),
    on='year', how='outer'
)

yearly_fall_chinook = yearly_fall_chinook[(yearly_fall_chinook['year'] >= 2000) & (yearly_fall_chinook['year'] <= 2022)]
yearly_fall_chinook['delta'] = yearly_fall_chinook['mcnary'] - yearly_fall_chinook['priest_rapids'] - yearly_fall_chinook['prosser'] - yearly_fall_chinook['ice_harbor']
yearly_fall_chinook['percent_delta'] = yearly_fall_chinook['delta'] / yearly_fall_chinook['mcnary']

In [ ]:
yearly_summer_chinook

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 5))
p = ax1.bar(yearly_spring_chinook['year'], yearly_spring_chinook['delta'], label='spring')
ax1.bar_label(p, label_type='edge', labels=yearly_spring_chinook['percent_delta'].round(2).astype(str), padding=3)

ax2.bar(yearly_spring_chinook['year'], yearly_spring_chinook['percent_delta'], label='spring')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 5))
p = ax1.bar(yearly_summer_chinook['year'], yearly_summer_chinook['delta'], label='summer')
ax1.bar_label(p, label_type='edge', labels=yearly_summer_chinook['percent_delta'].round(2).astype(str), padding=3)

ax2.bar(yearly_summer_chinook['year'], yearly_summer_chinook['percent_delta'], label='summer')


In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 5))
p = ax1.bar(yearly_fall_chinook['year'], yearly_fall_chinook['delta'], label='fall')
ax1.bar_label(p, label_type='edge', labels=yearly_fall_chinook['percent_delta'].round(2).astype(str), padding=3)

ax2.bar(yearly_fall_chinook['year'], yearly_fall_chinook['percent_delta'], label='fall')

In [ ]:
fig, ax = plt.subplots()
ax.bar(yearly_spring_chinook['year'], yearly_spring_chinook['delta'], label='McNary')
ax.bar(yearly_summer_chinook['year'], yearly_summer_chinook['delta'], label='Priest Rapids')

In [ ]:
# data from https://allisonhorst.github.io/palmerpenguins/

import matplotlib.pyplot as plt
import numpy as np

species = yearly_spring_chinook['year']
penguin_means = {
    'Spring': yearly_spring_chinook['delta'],
    'Summer': yearly_summer_chinook['delta'],
    'Fall': yearly_fall_chinook['delta'],
}

x = np.arange(len(species))  # the label locations
width = .25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(figsize=(15, 5))

for attribute, measurement in penguin_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    # ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Fish count')
ax.set_title('Change in Chinook population in the Hanford Reach')
ax.set_xticks(x + width, species)
ax.legend(loc='upper left', ncols=3)
# ax.set_ylim(0, 250)

plt.show()

In [ ]:
# data from https://allisonhorst.github.io/palmerpenguins/

import matplotlib.pyplot as plt
import numpy as np

species = yearly_spring_chinook['year']
penguin_means = {
    'Spring': yearly_spring_chinook['percent_delta'],
    'Summer': yearly_summer_chinook['percent_delta'],
    'Fall': yearly_fall_chinook['percent_delta'],
}

x = np.arange(len(species))  # the label locations
width = .25  # the width of the bars
multiplier = 0

fig, ax = plt.subplots(figsize=(15, 5))

for attribute, measurement in penguin_means.items():
    offset = width * multiplier
    rects = ax.bar(x + offset, measurement, width, label=attribute)
    # ax.bar_label(rects, padding=3)
    multiplier += 1

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Fish count')
ax.set_title('Change in Chinook population in the Hanford Reach as a percentage of Fish Count at McNary Dam')
ax.set_xticks(x + width, species)
ax.legend(loc='upper left', ncols=3)
# ax.set_ylim(0, 250)

plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.bar(yearly_spring_chinook['year'], yearly_spring_chinook['percent_delta'], label='spring')

In [ ]:
fig, ax = plt.subplots()
ax.bar(yearly_summer_chinook['year'], yearly_summer_chinook['percent_delta'], label='summer')

In [ ]:
fig, ax = plt.subplots()
ax.bar(yearly_fall_chinook['year'], yearly_fall_chinook['percent_delta'], label='fall')


In [31]:
thorr_df = pd.read_csv("/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/02-Fish/data/thorr/thorr_temp.csv")
thorr_df['date'] = pd.to_datetime(thorr_df['Date'])

thorr_df = thorr_df[thorr_df['RiverID'] == 9].copy()

# Year 2000 Fall Chinook

In [32]:
fall_chinook_2000 = mcnary_chinook_df[(mcnary_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'mcnary'}).merge(
    priest_rapids_chinook_df[(priest_rapids_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'priest_rapids'}),
    on='date', how='outer'
).merge(
    prosser_chinook_df[(prosser_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'prosser'}),
    on='date', how='outer'
).merge(
    ice_harbor_chinook_df[(ice_harbor_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'ice_harbor'}),
    on='date', how='outer'
)

# fall_chinook_2000 = fall_chinook_2000[(fall_chinook_2000['date'].dt.year >= 2000) & (fall_chinook_2000['date'].dt.year <= 2022)]
fall_chinook_2000 = fall_chinook_2000[(fall_chinook_2000['date'].dt.year == 2000)]
fall_chinook_2000.fillna(0, inplace=True)
fall_chinook_2000['delta'] = fall_chinook_2000['mcnary'] - fall_chinook_2000['priest_rapids'] - fall_chinook_2000['prosser'] - fall_chinook_2000['ice_harbor']
fall_chinook_2000['percent_delta'] = fall_chinook_2000['delta'] / fall_chinook_2000['mcnary']

In [33]:
fall_chinook_2000 = mcnary_chinook_df[(mcnary_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'mcnary'}).merge(
    priest_rapids_chinook_df[(priest_rapids_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'priest_rapids'}),
    on='date', how='outer'
).merge(
    prosser_chinook_df[(prosser_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'prosser'}),
    on='date', how='outer'
).merge(
    ice_harbor_chinook_df[(ice_harbor_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'ice_harbor'}),
    on='date', how='outer'
)

# fall_chinook_2000 = fall_chinook_2000[(fall_chinook_2000['date'].dt.year >= 2000) & (fall_chinook_2000['date'].dt.year <= 2022)]
fall_chinook_2000 = fall_chinook_2000[(fall_chinook_2000['date'].dt.year == 2000)]
fall_chinook_2000.fillna(0, inplace=True)
fall_chinook_2000['delta'] = fall_chinook_2000['mcnary'] - fall_chinook_2000['priest_rapids'] - fall_chinook_2000['prosser'] - fall_chinook_2000['ice_harbor']
fall_chinook_2000['percent_delta'] = fall_chinook_2000['delta'] / fall_chinook_2000['mcnary']

In [34]:
test = fall_chinook_2000.resample('W', on='date').sum().reset_index()

In [35]:
test['population'] = test['delta'].cumsum()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
test[test['date'].dt.year == 2000].plot(x='date', y='mcnary', ax=ax, label='Count at McNary')
test[test['date'].dt.year == 2000].plot(x='date', y='population', ax=ax, label='Chinook Population')
test[test['date'].dt.year == 2000].plot(x='date', y='delta', ax=ax, label='$\Delta$ Chinook Population')
ax.axhline(0, color='black', linewidth=1)
ax.set_title('Fall Chinook population at Hanford Reach in 2000')

In [38]:
closeup_year = 2000

# thorr_df = pd.read_csv("/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/02-Fish/Data/database/thorr_data.csv")
# thorr_df['date'] = pd.to_datetime(thorr_df['Date'])

# thorr_df = thorr_df[thorr_df['RiverID'] == 9].copy()
thorr_df_closeup = thorr_df[(thorr_df['date'] >= f'{closeup_year}-08-01') & (thorr_df['date'] <= f'{closeup_year}-11-15')]

In [39]:
grouped_closeup = thorr_df_closeup.groupby('RKm')

merged_thorr = pd.DataFrame(columns=['date'])
for group in grouped_closeup:
    resampled = group[1].resample('W', on='date').mean(numeric_only=True).reset_index()
    # print(resampled)
    resampled.rename(columns={'EstTempC': resampled['RKm'].iloc[0]}, inplace=True)
    merged_thorr = pd.merge(merged_thorr, resampled[['date', resampled['RKm'].iloc[0]]], on='date', how='outer')

merged_thorr['date'] = pd.to_datetime(merged_thorr['date'])

#make date the index
merged_thorr.set_index('date', inplace=True)
# sort by date
merged_thorr.sort_index(inplace=True)

In [40]:
hanford_reach_temp = merged_thorr[range(460, 630, 10)].reset_index()

In [ ]:
hanford_reach_temp['mean_temp'] = hanford_reach_temp.drop(columns=['date']).mean(axis=1)
hanford_reach_temp[460]

In [ ]:
test

In [ ]:
hanford_reach_temp

In [ ]:
fig, (ax, ax1) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
test[test['date'].dt.year == 2000].plot(x='date', y='mcnary', ax=ax, label='Count at McNary')
test[test['date'].dt.year == 2000].plot(x='date', y='population', ax=ax, label='Chinook Population')
test[test['date'].dt.year == 2000].plot(x='date', y='delta', ax=ax, label='$\Delta$ Chinook Population')
ax.axhline(0, color='black', linewidth=1)
ax.set_title('Fall Chinook population at Hanford Reach in 2000')

# ax1 = ax.twinx()
hanford_reach_temp.plot(x='date', y='mean_temp', ax=ax1, label='Mean Temperature')
# hanford_reach_temp.plot(x='date', y=630, ax=ax1, label='630')
# ax1.axvline(pd.Timestamp(f'2000-10-15'), color='red', linestyle='--')
# ax.axvline(pd.Timestamp(f'2000-10-15'), color='red', linestyle='--')

ax1.axhline(20, color='black', linestyle='--')

In [ ]:
a = hanford_reach_temp[['date', 'mean_temp']].merge(test[test['date'].dt.year == 2000], on='date')
fig, ax = plt.subplots()
ax.scatter(a['mean_temp'], a['delta'],)

# Year 2001 Fall Chinook

In [46]:
fall_chinook_2001 = mcnary_chinook_df[(mcnary_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'mcnary'}).merge(
    priest_rapids_chinook_df[(priest_rapids_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'priest_rapids'}),
    on='date', how='outer'
).merge(
    prosser_chinook_df[(prosser_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'prosser'}),
    on='date', how='outer'
).merge(
    ice_harbor_chinook_df[(ice_harbor_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'ice_harbor'}),
    on='date', how='outer'
)

# fall_chinook_2001 = fall_chinook_2001[(fall_chinook_2001['date'].dt.year >= 2001) & (fall_chinook_2001['date'].dt.year <= 2022)]
fall_chinook_2001 = fall_chinook_2001[(fall_chinook_2001['date'].dt.year == 2001)]
fall_chinook_2001.fillna(0, inplace=True)
fall_chinook_2001['delta'] = fall_chinook_2001['mcnary'] - fall_chinook_2001['priest_rapids'] - fall_chinook_2001['prosser'] - fall_chinook_2001['ice_harbor']
fall_chinook_2001['percent_delta'] = fall_chinook_2001['delta'] / fall_chinook_2001['mcnary']

In [47]:
fall_chinook_2001 = mcnary_chinook_df[(mcnary_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'mcnary'}).merge(
    priest_rapids_chinook_df[(priest_rapids_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'priest_rapids'}),
    on='date', how='outer'
).merge(
    prosser_chinook_df[(prosser_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'prosser'}),
    on='date', how='outer'
).merge(
    ice_harbor_chinook_df[(ice_harbor_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'ice_harbor'}),
    on='date', how='outer'
)

# fall_chinook_2001 = fall_chinook_2001[(fall_chinook_2001['date'].dt.year >= 2001) & (fall_chinook_2001['date'].dt.year <= 2022)]
fall_chinook_2001 = fall_chinook_2001[(fall_chinook_2001['date'].dt.year == 2001)]
fall_chinook_2001.fillna(0, inplace=True)
fall_chinook_2001['delta'] = fall_chinook_2001['mcnary'] - fall_chinook_2001['priest_rapids'] - fall_chinook_2001['prosser'] - fall_chinook_2001['ice_harbor']
fall_chinook_2001['percent_delta'] = fall_chinook_2001['delta'] / fall_chinook_2001['mcnary']

In [48]:
test = fall_chinook_2001.resample('W', on='date').sum().reset_index()

In [49]:
test['population'] = test['delta'].cumsum()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
test[test['date'].dt.year == 2001].plot(x='date', y='mcnary', ax=ax, label='Count at McNary')
test[test['date'].dt.year == 2001].plot(x='date', y='population', ax=ax, label='Chinook Population')
test[test['date'].dt.year == 2001].plot(x='date', y='delta', ax=ax, label='$\Delta$ Chinook Population')
ax.axhline(0, color='black', linewidth=1)
ax.set_title('Fall Chinook population at Hanford Reach in 2001')

In [51]:
closeup_year = 2001

# thorr_df = pd.read_csv("/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/02-Fish/Data/database/thorr_data.csv")
# thorr_df['date'] = pd.to_datetime(thorr_df['Date'])

thorr_df = thorr_df[thorr_df['RiverID'] == 9].copy()
thorr_df_closeup = thorr_df[(thorr_df['date'] >= f'{closeup_year}-08-01') & (thorr_df['date'] <= f'{closeup_year}-11-15')]

In [52]:
grouped_closeup = thorr_df_closeup.groupby('RKm')

merged_thorr = pd.DataFrame(columns=['date'])
for group in grouped_closeup:
    resampled = group[1].resample('W', on='date').mean(numeric_only=True).reset_index()
    # print(resampled)
    resampled.rename(columns={'EstTempC': resampled['RKm'].iloc[0]}, inplace=True)
    merged_thorr = pd.merge(merged_thorr, resampled[['date', resampled['RKm'].iloc[0]]], on='date', how='outer')

merged_thorr['date'] = pd.to_datetime(merged_thorr['date'])

#make date the index
merged_thorr.set_index('date', inplace=True)
# sort by date
merged_thorr.sort_index(inplace=True)

In [53]:
hanford_reach_temp = merged_thorr[range(460, 630, 10)].reset_index()

In [ ]:
hanford_reach_temp['mean_temp'] = hanford_reach_temp.drop(columns=['date']).mean(axis=1)
hanford_reach_temp[460]

In [ ]:
test

In [ ]:
fig, (ax, ax1) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
test[test['date'].dt.year == 2001].plot(x='date', y='mcnary', ax=ax, label='Count at McNary')
test[test['date'].dt.year == 2001].plot(x='date', y='population', ax=ax, label='Chinook Population')
test[test['date'].dt.year == 2001].plot(x='date', y='delta', ax=ax, label='$\Delta$ Chinook Population')
ax.axhline(0, color='black', linewidth=1)
ax.set_title('Fall Chinook population at Hanford Reach in 2001')

# ax1 = ax.twinx()
hanford_reach_temp.plot(x='date', y='mean_temp', ax=ax1, label='Mean Temperature')
# hanford_reach_temp.plot(x='date', y=630, ax=ax1, label='630')
# ax1.axvline(pd.Timestamp(f'2001-10-15'), color='red', linestyle='--')
# ax.axvline(pd.Timestamp(f'2001-10-15'), color='red', linestyle='--')

ax1.axhline(20, color='black', linestyle='--')

In [ ]:
a = hanford_reach_temp[['date', 'mean_temp']].merge(test[test['date'].dt.year == 2001], on='date')
fig, ax = plt.subplots()
ax.scatter(a['mean_temp'], a['delta'],)

## General Chinook

In [58]:
hanford_chinook_df = mcnary_chinook_df[(mcnary_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'mcnary'}).merge(
    priest_rapids_chinook_df[(priest_rapids_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'priest_rapids'}),
    on='date', how='outer'
).merge(
    prosser_chinook_df[(prosser_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'prosser'}),
    on='date', how='outer'
).merge(
    ice_harbor_chinook_df[(ice_harbor_chinook_df['run'] == 'fall')][['date', 'chinook']].rename(columns={'chinook': 'ice_harbor'}),
    on='date', how='outer'
)
hanford_chinook_df['year'] = hanford_chinook_df['date'].dt.year
hanford_chinook_df[['mcnary', 'priest_rapids', 'prosser', 'ice_harbor']] =hanford_chinook_df[['mcnary', 'priest_rapids', 'prosser', 'ice_harbor']].copy().fillna(0)
hanford_chinook_df['delta'] = hanford_chinook_df['mcnary'] - hanford_chinook_df['priest_rapids'] - hanford_chinook_df['prosser'] - hanford_chinook_df['ice_harbor']

# hanford_chinook_df['percent_delta'] = hanford_chinook_df['delta'] / hanford_chinook_df['mcnary']


In [ ]:
hanford_chinook_df

In [ ]:
for year in hanford_chinook_df['year'].unique():
    _hanford_chinook_df = hanford_chinook_df[hanford_chinook_df['date'].dt.year == year].copy()
    _hanford_chinook_df = _hanford_chinook_df[_hanford_chinook_df['date'] <= pd.to_datetime(f'{int(year)}-11-15')]
    _hanford_chinook_df = _hanford_chinook_df.resample('W', on='date').sum().reset_index()
    _hanford_chinook_df.drop(columns=['year'], inplace=True)
    _hanford_chinook_df['population'] = _hanford_chinook_df['delta'].cumsum()
    _hanford_chinook_df['norm_delta'] = _hanford_chinook_df['delta'] / _hanford_chinook_df['population'].max()
    # print(_hanford_chinook_df)

    fig, (ax, ax1) = plt.subplots(2, 1, figsize=(7, 7), sharex=True)
    _hanford_chinook_df.plot(x='date', y='mcnary', ax=ax, label='Count at McNary')
    _hanford_chinook_df.plot(x='date', y='population', ax=ax, label='Chinook Population')
    _hanford_chinook_df.plot(x='date', y='delta', ax=ax, label='$\Delta$ Chinook Population')

    _thorr_df_closeup = thorr_df[(thorr_df['date'] >= _hanford_chinook_df['date'].min()) & (thorr_df['date'] <= _hanford_chinook_df['date'].max())]

    grouped_closeup = _thorr_df_closeup.groupby('RKm')

    merged_thorr = pd.DataFrame(columns=['date'])
    for group in grouped_closeup:
        resampled = group[1].resample('W', on='date').mean(numeric_only=True).reset_index()
        # print(resampled)
        resampled.rename(columns={'EstTempC': resampled['RKm'].iloc[0]}, inplace=True)
        merged_thorr = pd.merge(merged_thorr, resampled[['date', resampled['RKm'].iloc[0]]], on='date', how='outer')

    merged_thorr['date'] = pd.to_datetime(merged_thorr['date'])
    #make date the index
    merged_thorr.set_index('date', inplace=True)
    # sort by date
    merged_thorr.sort_index(inplace=True)

    hanford_reach_temp = merged_thorr[range(460, 630, 10)].reset_index()
    hanford_reach_temp['mean_temp'] = hanford_reach_temp.drop(columns=['date']).mean(axis=1)

    hanford_reach_temp.plot(x='date', y='mean_temp', ax=ax1, label='Mean Temperature')

    ax.set_ylabel('Count')
    ax1.set_ylabel('Temperature (C)')

    ax.axhline(0, color='black', linewidth=1)
    ax1.axhline(20, color='black', linestyle='--')

    fig.suptitle(f'Fall Chinook population at Hanford Reach in {int(year)}')
    fig.tight_layout()
    # break

In [ ]:
hanford_chinook_df

In [ ]:
hanford_chinook_df[hanford_chinook_df['date'].dt.year == 2000]

In [ ]:
_hanford_chinook_df = hanford_chinook_df.resample('W', on='date').sum().reset_index()
_hanford_chinook_df

In [34]:
grouped_closeup = thorr_df.groupby('RKm')

_merged_thorr = pd.DataFrame(columns=['date'])
for group in grouped_closeup:
    resampled = group[1].resample('W', on='date').mean(numeric_only=True).reset_index()
    # print(resampled)
    resampled.rename(columns={'EstTempC': resampled['RKm'].iloc[0]}, inplace=True)
    _merged_thorr = pd.merge(_merged_thorr, resampled[['date', resampled['RKm'].iloc[0]]], on='date', how='outer')

_merged_thorr['date'] = pd.to_datetime(_merged_thorr['date'])

#make date the index
_merged_thorr.set_index('date', inplace=True)
# sort by date
_merged_thorr.sort_index(inplace=True)

In [35]:
_hanford_temp_df = _merged_thorr[range(460, 630, 10)].reset_index()
_hanford_temp_df['mean_temp'] = _hanford_temp_df.drop(columns=['date']).mean(axis=1)

In [36]:
_hanford_chinook_df = _hanford_chinook_df.merge(_hanford_temp_df, on='date', how='left')

In [37]:
_hanford_chinook_df = allocate_runs(_hanford_chinook_df, run_schedules["mcnary"])

In [ ]:
_hanford_chinook_df['chinook_population'] = np.nan
_hanford_chinook_df.head()

In [43]:
for year in _hanford_chinook_df['year'].unique():
    _hanford_chinook_df.loc[_hanford_chinook_df['year'] == year, 'chinook_population'] = _hanford_chinook_df.loc[_hanford_chinook_df['year'] == year, 'delta'].cumsum()

In [52]:
_hanford_chinook_df['date'] = pd.to_datetime(_hanford_chinook_df['date'])

In [ ]:
selected_year = 2013

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(10, 7))
_hanford_chinook_df[(_hanford_chinook_df['year'] == selected_year) & (_hanford_chinook_df['run'] == 'fall')].plot(x='date', y='mcnary', ax=ax1, label='Count at McNary')
_hanford_chinook_df[(_hanford_chinook_df['year'] == selected_year) & (_hanford_chinook_df['run'] == 'fall')].plot(x='date', y='chinook_population', ax=ax1, label='Chinook Population')
_hanford_chinook_df[(_hanford_chinook_df['year'] == selected_year) & (_hanford_chinook_df['run'] == 'fall')].plot(x='date', y='delta', ax=ax1, label='$\Delta$ Chinook Population')
ax1.axhline(0, color='black', linewidth=1)

_hanford_chinook_df[(_hanford_chinook_df['year'] == selected_year) & (_hanford_chinook_df['run'] == 'fall')].plot(x='date', y='mean_temp', ax=ax2, label='Mean Temperature')
ax2.axhline(20, color='black', linestyle='--')
fig.tight_layout()

In [ ]:
for year in _hanford_chinook_df['year'].unique():
    selected_year = year
    _hanford_chinook_df['date'] = pd.to_datetime(_hanford_chinook_df['date'])

    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(9, 7))
    _hanford_chinook_df[(_hanford_chinook_df['year'] == selected_year) & (_hanford_chinook_df['run'] == 'fall')].plot(x='date', y='mcnary', ax=ax1, label='Count at McNary')
    _hanford_chinook_df[(_hanford_chinook_df['year'] == selected_year) & (_hanford_chinook_df['run'] == 'fall')].plot(x='date', y='chinook_population', ax=ax1, label='Chinook Population')
    _hanford_chinook_df[(_hanford_chinook_df['year'] == selected_year) & (_hanford_chinook_df['run'] == 'fall')].plot(x='date', y='delta', ax=ax1, label='$\Delta$ Chinook Population')
    ax1.axhline(0, color='black', linewidth=1)
    ax1.set_title(f'Fall Chinook population at Hanford Reach in {selected_year}')

    _hanford_chinook_df[(_hanford_chinook_df['year'] == selected_year) & (_hanford_chinook_df['run'] == 'fall')].plot(x='date', y='mean_temp', ax=ax2, label='Mean Temperature')
    ax2.axhline(20, color='black', linestyle='--')
    
    fig.tight_layout()
    break

In [ ]:
fig, ax = plt.subplots()
_hanford_chinook_df[_hanford_chinook_df['run']== 'fall'].plot.scatter(x='mean_temp', y='percent_delta', ax=ax
# , c='year', cmap='viridis'
)
ax.set_xlabel('Mean Temperature (C)')
ax.set_ylabel('Change in Chinook Population')

In [ ]:
fig, ax = plt.subplots()
_hanford_chinook_df[_hanford_chinook_df['run']== 'fall'].plot.scatter(x='mean_temp', y='delta', ax=ax, c='year', cmap='viridis')
ax.set_xlabel('Mean Temperature (C)')
ax.set_ylabel('Change in Chinook Population')

In [ ]:
_hanford_chinook_df[
    (_hanford_chinook_df["run"] == "fall") & (_hanford_chinook_df["year"] == 2000)
].plot(
    x="mean_temp",
    y="delta",
    #  ax=ax,
    # c="year",
    # cmap="viridis",
)

In [ ]:
_grouped = hanford_chin

In [ ]:
# for each year, find the population retained (cumsum of delta)

for year in _hanford_chinook_df['year'].unique():
    _hanford_chinook_df[_hanford_chinook_df['year']['population']] = _hanford_chinook_df[_hanford_chinook_df['year'] == year]['delta'].cumsum()

In [ ]:
fig, (ax, ax1) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
_hanford_chinook_df[_hanford_chinook_df['date'].dt.year == 2001].plot(x='date', y='mcnary', ax=ax, label='Count at McNary')
_hanford_chinook_df[_hanford_chinook_df['date'].dt.year == 2001].plot(x='date', y='population', ax=ax, label='Chinook Population')
_hanford_chinook_df[_hanford_chinook_df['date'].dt.year == 2001].plot(x='date', y='delta', ax=ax, label='$\Delta$ Chinook Population')
ax.axhline(0, color='black', linewidth=1)
ax.set_title('Fall Chinook population at Hanford Reach in 2001')

# ax1 = ax.twinx()
hanford_reach_temp.plot(x='date', y='mean_temp', ax=ax1, label='Mean Temperature')
# hanford_reach_temp.plot(x='date', y=630, ax=ax1, label='630')
# ax1.axvline(pd.Timestamp(f'2001-10-15'), color='red', linestyle='--')
# ax.axvline(pd.Timestamp(f'2001-10-15'), color='red', linestyle='--')

ax1.axhline(20, color='black', linestyle='--')

In [ ]:
_hanford_chinook_df[['mean_temp', 'delta']].cov()

In [303]:
temp_duration_curve = pd.DataFrame()
temp_duration_curve['mean_temp'] = _hanford_temp_df['mean_temp']

temp_duration_curve.sort_values('mean_temp', inplace=True)
temp_duration_curve['Rank'] = temp_duration_curve['mean_temp'].rank(ascending=False)

temp_duration_curve['probability'] = temp_duration_curve['Rank'] / len(temp_duration_curve)

In [ ]:
fig, ax = plt.subplots()
ax.plot(temp_duration_curve['probability'], temp_duration_curve['mean_temp'], )
# log axis for y
# ax.set_yscale('log')
ax.set_xlabel('Exceedance Probability')
ax.set_ylabel('Temperature (C)')

In [ ]:
fig, ax = plt.subplots()
ax.scatter(_test['mean_temp'], _test['delta'])